In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV,StratifiedKFold
from sklearn.model_selection import cross_val_score

In [3]:
df= pd.read_csv("train_preprocessed.csv",index_col = 0)
#df_test = pd.read_csv("test_preprocessed.csv",index_col = 0)

In [4]:
df.head()

,Survived,Pclass,Sex,Embarked,ischild,hasCabin,Hasfam
PassengerId,,,,,,,
1,0,3,0,1,0,0,1
2,1,1,1,1,0,1,1
3,1,3,1,1,0,0,0
4,1,1,1,1,0,1,1
5,0,3,0,1,0,0,0


In [5]:
X = df.drop(["Survived"],axis=1)

In [6]:
y =df["Survived"]

In [7]:
X.head()

,Pclass,Sex,Embarked,ischild,hasCabin,Hasfam
PassengerId,,,,,,
1,3,0,1,0,0,1
2,1,1,1,0,1,1
3,3,1,1,0,0,0
4,1,1,1,0,1,1
5,3,0,1,0,0,0


In [8]:
y.head()

PassengerId
1    0
2    1
3    1
4    1
5    0
Name: Survived, dtype: int64

In [9]:
X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=0.1)

In [10]:
neigh =  KNeighborsClassifier()
lin= LogisticRegression() 
vect= SVC()
tree = DecisionTreeClassifier()

In [74]:
#lname=["neighbors","linear","vector","tree"]
#lname=["neighbors","vector","tree"]
lname=["linear","vector","tree"]

In [75]:
lclf= [neigh,lin,vect,tree]
#lclf= [neigh,vect,tree]
lclf= [lin,vect,tree]

In [13]:
# 4 grilles d'hyperparamètres

In [76]:
param_list = [#{"n_neighbors": [2,3,5,7,9], "weight": ['uniform', 'distance']},
              {"solver" : ['newton-cg', 'lbfgs', 'liblinear']},
              {"kernel":['linear', 'poly', 'rbf', 'sigmoid'],'C': [0.1, 1, 10, 100],'gamma':['scale', 'auto']},
              {'criterion' : ["gini", "entropy"],'splitter' :["best", "random"]}]


In [77]:
gridcvs = {}

In [78]:
for Name,Clf,Param in zip(lname,lclf,param_list):
    gridcvs[Name] = GridSearchCV(Clf,Param)
    #print(Name,Clf,Param)
    

In [79]:
strat = StratifiedKFold(n_splits=5)

In [80]:
outerscore = {}

In [81]:
for name,gcv in gridcvs.items():
    nested_score = cross_val_score(gcv,X_train,y_train,cv=strat)
    outerscore[name] = nested_score
    print(f"{name}: outer accuracy {100 * nested_score.mean():.2f} +/- { 100 * nested_score.std():.2f}")

linear: outer accuracy 78.90 +/- 1.58
vector: outer accuracy 81.77 +/- 2.21
tree: outer accuracy 81.90 +/- 2.09


In [90]:
#selection du meilleur modèle
# tree  décision modèle

In [ ]:
#Hyperparamètres

In [91]:
param_grid = {'criterion' : ["gini", "entropy"],
              'splitter' :["best", "random"],
              "max_features" : [None,"auto","sqrt","log2"]}

In [92]:
gridcv = GridSearchCV(estimator=tree,param_grid=param_grid)

In [93]:
gridcv.fit(X_train,y_train)

GridSearchCV(estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_features': [None, 'auto', 'sqrt', 'log2'],
                         'splitter': ['best', 'random']})

In [94]:
gridcv.best_params_

{'criterion': 'gini', 'max_features': None, 'splitter': 'best'}

In [ ]:
#vérification

In [95]:
tree2 = DecisionTreeClassifier(criterion="gini",max_features=None,splitter="best")

In [96]:
tree2.fit(X_train,y_train)

DecisionTreeClassifier()

In [97]:
y_pred= tree2.predict(X_val)

In [98]:
pd.crosstab(y_val,y_pred)

col_0,0,1
Survived,,
0,49,4
1,18,19


In [99]:
from sklearn.metrics import classification_report

In [101]:
print(classification_report(y_val,y_pred))

              precision    recall  f1-score   support

           0       0.73      0.92      0.82        53
           1       0.83      0.51      0.63        37

    accuracy                           0.76        90
   macro avg       0.78      0.72      0.72        90
weighted avg       0.77      0.76      0.74        90

